<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_Track_Learning/blob/main/CV/tutorial/%5BCV_realtime%5D5.%20torchvision_finetuning_instance_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


# TorchVision Object Detection Finetuning Tutorial



본 튜토리얼에서는 Penn-Fudan Database for Pedestrian Detection and Segmentation 데이터셋으로 미리 학습된 Mask R-CNN 모델을 미세조정 해 볼 것입니다. 이 데이터셋에는 보행자 인스턴스(instance, 역자주: 이미지 내에서 사람의 위치 좌표와 픽셀 단위의 사람 여부를 구분한 정보를 포함합니다.) 345명이 있는 170개의 이미지가 포함되어 있으며, 우리는 이 이미지를 사용하여 사용자 정의 데이터셋에 인스턴스 분할(Instance Segmentation) 모델을 학습하기 위해 torchvision의 새로운 기능을 사용하는 방법을 설명 할 예정입니다.



## Defining the Dataset

객체 검출, 인스턴스 분할 및 사용자 키포인트(Keypoint) 검출을 학습하기 위한 참조 스크립트를 통해 새로운 사용자 정의 데이터셋 추가를 쉽게 진행해 볼 수 있습니다. 데이터셋은 표준 torch.utils.data.Dataset 클래스를 상속 받아야 하며, __len__ 와 __getitem__ 메소드를 구현해 주어야 합니다.

데이터셋에서 필요한 유일한 특성은 __getitem__ 메소드가 다음을 반환 해야 하는 것입니다:

> ```이미지``` : PIL(Python Image Library) 이미지의 크기 (H, W)

> ```대상```: 다음의 필드를 포함하는 사전 타입

> ```boxes (FloatTensor[N, 4])``` : N 개의 바운딩 박스(Bounding box)의 좌표를 [x0, y0, x1, y1] 형태로 가집니다. x와 관련된 값 범위는 0 부터 W 이고 y와 관련된 값의 범위는 0 부터 H 까지입니다.

> ```labels (Int64Tensor[N])``` : 바운딩 박스 마다의 라벨 정보입니다. 0 은 항상 배경의 클래스를 표현합니다.

> ```image_id (Int64Tensor[1])```: 이미지 구분자입니다. 데이터셋의 모든 이미지 간에 고유한 값이어야 하며 평가 중에도 사용됩니다.

> ```area (Tensor[N])```: 바운딩 박스의 면적입니다. 면적은 평가 시 작음,중간,큰 박스 간의 점수를 내기 위한 기준이며 COCO 평가를 기준으로 합니다.

> ```iscrowd (UInt8Tensor[N])```: 이 값이 참일 경우 평가에서 제외합니다.

> ```(선택적) masks (UInt8Tensor[N, H, W])```: N 개의 객체 마다의 분할 마스크 정보입니다.

> ```(선택적) keypoints (FloatTensor[N, K, 3])```: N 개의 객체마다의 키포인트 정보입니다. 키포인트는 [x, y, visibility] 형태의 값입니다. visibility 값이 0인 경우 키포인트는 보이지 않음을 의미합니다. 데이터 증강(Data augmentation)의 경우 키포인트 좌우 반전의 개념은 데이터 표현에 따라 달라지며, 새로운 키포인트 표현에 대해 《references/detection/transforms.py》 코드 부분을 수정 해야 할 수도 있습니다.

모델이 위의 방법대로 리턴을 하면, 학습과 평가 둘 다에 대해서 동작을 할 것이며 평가 스크립트는 pip install pycocotools` 로 설치 가능한 ```pycocotools``` 를 사용하게 될 것입니다.



### labels 에 대한 참고사항.

이 모델은 클래스 0 을 배경으로 취급합니다. 만약 준비한 데이터셋에 배경의 클래스가 없다면, labels 에도 0 이 없어야 합니다. 예를 들어, 고양이 와 강아지 의 오직 2개의 클래스만 분류한다고 가정하면, (0 이 아닌) 1 이 고양이 를, 2 가 강아지 를 나타내도록 정의해야 합니다. 따라서, 이 예시에서, 어떤 이미지에 두 개의 클래스를 모두 있다면, labels 텐서는 [1,2] 와 같은 식이 되어야 합니다.

추가로, 학습 중에 가로 세로 비율 그룹화를 사용하려는 경우(각 배치에 유사한 가로 세로 비율이 있는 영상만 포함되도록), 이미지의 넓이, 높이를 리턴할 수 있도록 get_height_and_width 메소드를 구현하기를 추천합니다. 이 메소드가 구현되지 않은 경우에는 모든 데이터셋은 __getitem__ 를 통해 메모리에 이미지가 로드되며 사용자 정의 메소드를 제공하는 것보다 느릴 수 있습니다.

### Writing a custom dataset for PennFudan

PennFudan 데이터셋을 위한 코드를 작성해 보겠습니다. `다운로드 후 압축 파일을 해제하면<https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip>`__, 다음의 폴더 구조를 볼 수 있습니다:
::
```
   PennFudanPed/
     PedMasks/
       FudanPed00001_mask.png
       FudanPed00002_mask.png
       FudanPed00003_mask.png
       FudanPed00004_mask.png
       ...
     PNGImages/
       FudanPed00001.png
       FudanPed00002.png
       FudanPed00003.png
       FudanPed00004.png
```



In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬하여
        # 이미지와 분할 마스크 정렬을 확인합니다
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # 이미지와 마스크를 읽어옵니다
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        # 분할 마스크는 RGB로 변환하지 않음을 유의하세요
        # 왜냐하면 각 색상은 다른 인스턴스에 해당하며, 0은 배경에 해당합니다
        img = read_image(img_path)
        mask = read_image(mask_path)

        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눕니다
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # 객체 종류는 한 종류만 존재합니다. 예제에서는 사람만이 대상입니다
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## Defining your model

Torchvision model zoo, (미리 학습된 모델들을 모아 놓은 공간)에서 사용 가능한 모델들 중 하나를 이용해 모델을 수정하려면 보통 두가지 상황이 있습니다. 첫 번째 방법은 미리 학습된 모델에서 시작해서 마지막 레이어 수준만 미세 조정하는 것입니다. 다른 하나는 모델의 백본을 다른 백본으로 교체하는 것입니다.

역자주: 백본 모델을 ResNet101 에서 MobilenetV2 로 교체하면 수행 속도 향상을 기대할 수 있습니다. 대신 인식 성능은 저하 될 수 있습니다.


### 1 - Finetuning from a pretrained model

COCO에 대해 미리 학습된 모델에서 시작하여 특정 클래스를 위해 미세 조정을 원한다고 가정해 봅시다. 아래와 같은 방법으로 가능합니다:



In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# COCO로 미리 학습된 모델 읽기
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# 분류기를 새로운 것으로 교체하는데, num_classes는 사용자가 정의합니다
num_classes = 2  # 1 클래스(사람) + 배경

# 분류기에서 사용할 입력 특징의 차원 정보를 얻습니다
in_features = model.roi_heads.box_predictor.cls_score.in_features

# 미리 학습된 모델의 머리 부분을 새로운 것으로 교체합니다
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 68.7MB/s]


In [ ]:
model.roi_heads.box_predictor.cls_score.in_features

1024

### 2 - Modifying the model to add a different backbone



In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 분류 목적으로 미리 학습된 모델을 로드하고 특징들만을 리턴하도록 합니다
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# Faster RCNN은 백본의 출력 채널 수를 알아야 합니다.
# mobilenetV2의 경우 1280이므로 여기에 추가해야 합니다.
backbone.out_channels = 1280

# RPN(Region Proposal Network)이 5개의 서로 다른 크기와 3개의 다른 측면 비율(Aspect ratio)을 가진
# 5 x 3개의 앵커를 공간 위치마다 생성하도록 합니다.
# 각 특징 맵이 잠재적으로 다른 사이즈와 측면 비율을 가질 수 있기 때문에 Tuple[Tuple[int]] 타입을 가지도록 합니다.

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1, 2),))

# 관심 영역의 자르기 및 재할당 후 자르기 크기를 수행하는 데 사용할 피쳐 맵을 정의합니다.
# 만약 백본이 텐서를 리턴할때, featmap_names 는 [0] 이 될 것이라고 예상합니다.
# 일반적으로 백본은 OrderedDict[Tensor] 타입을 리턴해야 합니다.
# 그리고 특징맵에서 사용할 featmap_names 값을 정할 수 있습니다.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# 조각들을 Faster RCNN 모델로 합칩니다.
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 70.2MB/s]


In [ ]:
print(f"Model structure: {backbone}\n\n")

for name, param in backbone.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} \n")

Model structure: Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stat

### Object detection and instance segmentation model for PennFudan Dataset

우리의 경우, 데이터 세트가 매우 작기 때문에, 우리는 1번 접근법을 따를 것이라는 점을 고려하여 미리 학습된 모델에서 미세 조정하는 방식으로 진행 하겠습니다.

여기서 인스턴스 분할 마스크도 계산하기를 원하기 때문에 Mask R-CNN를 사용합니다:



In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# 여기는 Segmentation을 위한 함수
def get_model_instance_segmentation(num_classes):
    # COCO 에서 미리 학습된 인스턴스 분할 모델을 읽어옵니다.
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # 분류를 위한 입력 특징 차원을 얻습니다.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 미리 학습된 헤더를 새로운 것으로 바꿉니다
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 마스크 분류기를 위한 입력 특징들의 차원을 얻습니다
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # 마스크 예측기를 새로운 것으로 바꿉니다
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

그렇습니다. 이렇게 하면 모델 을 사용자 정의 데이터셋에서 학습하고 평가할 준비가 될 겁니다.

## Putting everything together

``references/detection/`` 폴더 내에 검출 모델들의 학습과 평과를 쉽게 하기 위한 도움 함수들이 있습니다. 여기서 ``references/detection/engine.py``, ``references/detection/utils.py``, ``references/detection/transforms.py`` 를 사용 할 것입니다. ``references/detection`` 아래의 모든 파일과 폴더들을 사용자의 폴더로 복사한 뒤 사용합니다.

데이터 증강 / 변환을 위한 도움 함수를 작성해 봅시다



In [ ]:
import zipfile

os.system("wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip")

def unzip_file(zip_path, extract_to):
    """
    zip_path: 압축 해제할 ZIP 파일의 경로
    extract_to: 압축을 풀 폴더의 경로
    """
    # 압축을 풀 폴더가 없으면 생성
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # ZIP 파일 열기
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # ZIP 파일 내용을 지정된 폴더에 압축 해제
        zip_ref.extractall(extract_to)

# 사용 예시
zip_file_path = '/content/PennFudanPed.zip'  # ZIP 파일 경로
destination_folder = '/content/data/'  # 압축 해제될 폴더 경로

unzip_file(zip_file_path, destination_folder)

In [ ]:
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)



# Testing ``forward()`` method (Optional)
# ---------------------------------------
#
# 데이터셋을 반복하기 전에, 샘플 데이터로 학습과 추론 시 모델이 예상대로 동작하는지 살펴보는 것이 좋습니다.
import utils

dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'loss_classifier': tensor(0.6722, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0497, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.6924, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0638, grad_fn=<DivBackward0>)}
{'boxes': tensor([[8.4757e-03, 8.4242e+00, 4.9148e+00, 2.2252e+01],
        [0.0000e+00, 1.8917e+01, 8.0957e+00, 3.1016e+01],
        [6.3913e+00, 1.7397e+01, 1.9412e+01, 3.0886e+01],
        [0.0000e+00, 2.8286e+01, 4.6500e+00, 4.4743e+01],
        [6.4574e+00, 2.9429e+01, 1.9176e+01, 4.2293e+01],
        [0.0000e+00, 1.8126e+02, 1.0366e+02, 3.0000e+02],
        [1.3265e+02, 2.1487e+02, 1.8455e+02, 2.6529e+02],
        [0.0000e+00, 0.0000e+00, 3.4116e+01, 2.9297e+01],
        [1.2075e+02, 8.3306e+01, 1.7144e+02, 1.3413e+02],
        [2.2907e+02, 8.1671e+01, 2.8034e+02, 1.3287e+02],
        [7.4338e+01, 1.7909e+02, 1.2436e+02, 2.2892e+02],
        [3.6386e+01, 1.6636e+02, 8.7388e+01, 2.1740e+02],
        [3.6069e+01, 2.3809e+0

### 모델 학습



In [ ]:
from engine import train_one_epoch, evaluate

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = PennFudanDataset('data/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('data/PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=utils.collate_fn
)



In [ ]:
next(iter(data_loader_test))

((tensor([[[0.1176, 0.1176, 0.1255,  ..., 0.1765, 0.2627, 0.1961],
           [0.1490, 0.1176, 0.1294,  ..., 0.1961, 0.6784, 0.7451],
           [0.1451, 0.1294, 0.1686,  ..., 0.7216, 0.6392, 0.5686],
           ...,
           [0.4824, 0.4863, 0.4902,  ..., 0.5725, 0.5647, 0.5922],
           [0.4824, 0.4902, 0.4941,  ..., 0.5725, 0.5647, 0.5647],
           [0.4863, 0.4902, 0.4980,  ..., 0.5569, 0.5608, 0.5373]],
  
          [[0.1725, 0.1725, 0.1804,  ..., 0.1882, 0.2667, 0.2000],
           [0.2039, 0.1725, 0.1843,  ..., 0.2078, 0.6824, 0.7490],
           [0.2118, 0.1961, 0.2353,  ..., 0.7333, 0.6431, 0.5725],
           ...,
           [0.4824, 0.4863, 0.4902,  ..., 0.5725, 0.5647, 0.5922],
           [0.4824, 0.4902, 0.4941,  ..., 0.5725, 0.5647, 0.5647],
           [0.4863, 0.4902, 0.4980,  ..., 0.5569, 0.5608, 0.5373]],
  
          [[0.1137, 0.1137, 0.1216,  ..., 0.1529, 0.2353, 0.1686],
           [0.1451, 0.1137, 0.1255,  ..., 0.1725, 0.6510, 0.7176],
           [0.1412, 0.

In [ ]:
# get the model using our helper function
# 모바일넷은 성능이 낮아 resnet 기반 모델로 다시 변경합니다.
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it for 5 epochs
num_epochs = 5

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 149MB/s]


Epoch: [0]  [ 0/60]  eta: 0:09:21  lr: 0.000090  loss: 4.3019 (4.3019)  loss_classifier: 0.7415 (0.7415)  loss_box_reg: 0.5109 (0.5109)  loss_mask: 2.9980 (2.9980)  loss_objectness: 0.0408 (0.0408)  loss_rpn_box_reg: 0.0107 (0.0107)  time: 9.3636  data: 0.5065  max mem: 2150
Epoch: [0]  [10/60]  eta: 0:01:07  lr: 0.000936  loss: 1.6386 (2.2515)  loss_classifier: 0.4847 (0.4875)  loss_box_reg: 0.2661 (0.3066)  loss_mask: 0.7777 (1.4244)  loss_objectness: 0.0235 (0.0270)  loss_rpn_box_reg: 0.0039 (0.0060)  time: 1.3466  data: 0.0524  max mem: 2655
Epoch: [0]  [20/60]  eta: 0:00:39  lr: 0.001783  loss: 1.1393 (1.5612)  loss_classifier: 0.2478 (0.3437)  loss_box_reg: 0.2642 (0.3027)  loss_mask: 0.4050 (0.8867)  loss_objectness: 0.0207 (0.0219)  loss_rpn_box_reg: 0.0046 (0.0062)  time: 0.5600  data: 0.0081  max mem: 2764
Epoch: [0]  [30/60]  eta: 0:00:25  lr: 0.002629  loss: 0.6338 (1.2369)  loss_classifier: 0.1090 (0.2594)  loss_box_reg: 0.2581 (0.2856)  loss_mask: 0.2261 (0.6681)  loss_ob

In [ ]:
import matplotlib.pyplot as plt

from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks

os.system("wget https://tutorials.pytorch.kr/_static/img/tv_tutorial/tv_image05.png")

image = read_image("/content/tv_image05.png")
eval_transform = get_transform(train=False)

model.eval()
with torch.no_grad():
    x = eval_transform(image)
    # convert RGBA -> RGB and move to device
    x = x[:3, ...].to(device)
    predictions = model([x, ])
    pred = predictions[0]


image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
image = image[:3, ...]
pred_labels = [f"pedestrian: {score:.3f}" for label, score in zip(pred["labels"], pred["scores"])]
pred_boxes = pred["boxes"].long()
output_image = draw_bounding_boxes(image, pred_boxes, pred_labels, colors="red")

masks = (pred["masks"] > 0.7).squeeze(1)
output_image = draw_segmentation_masks(output_image, masks, alpha=0.5, colors="blue")


plt.figure(figsize=(12, 12))
plt.imshow(output_image.permute(1, 2, 0))

KeyboardInterrupt: ignored